In [1]:
import pandas as pd

In [2]:
colunas_finais = ['DATA',
'HORA',
'NATUREZA_ACIDENTE',
'BAIRRO',
'BAIRRO_CRUZAMENTO',
'TIPO',
'AUTOMOVEL_ENVOLVIDO',
'MOTO_ENVOLVIDA',
'CICLO_MOTOR_ENVOLVIDO',
'CICLISTA_ENVOLVIDO',
'PEDESTRE_ENVOLVIDO',
'ONIBUS_ENVOLVIDO',
'CAMINHAO_ENVOLVIDO',
'VIATURA_ENVOLVIDA',
'OUTROS_ENVOLVIDO',
'NUM_VITIMAS',
'NUM_VITIMAS_FATAIS']
mapeamento_2019 = ['DATA',
'hora',
'natureza_acidente',
'bairro',
'bairro_cruzamento',
'tipo',
'auto',
'moto',
'ciclom',
'ciclista',
'pedestre',
'onibus',
'caminhao',
'viatura',
'outros',
'vitimas',
'vitimasfatais']
mapeamento_2020_2021 = ['data',
'hora',
'natureza_acidente',
'bairro',
'bairro_cruzamento',
'tipo',
'auto',
'moto',
'ciclom',
'ciclista',
'pedestre',
'onibus',
'caminhao',
'viatura',
'outros',
'vitimas',
'vitimasfatais']
mapeamento_2022 = ['data',
'hora',
'natureza',
'bairro',
'bairro_cruzamento',
'tipo',
'auto',
'moto',
'ciclom',
'ciclista',
'pedestre',
'onibus',
'caminhao',
'viatura',
'outros',
'vitimas',
'vitimasfatais']

In [3]:
def carregar_df(caminho_arquivo, colunas_alvo):
    return pd.read_csv(caminho_arquivo, usecols=colunas_alvo, sep=';')

def processar_df(ano, caminho_arquivo, colunas_alvo, colunas_finais):
    df_ano = carregar_df(caminho_arquivo, colunas_alvo)
    df_ano.columns = colunas_finais
    return df_ano  

In [4]:
pre_processamento = {
    2019:{ 'caminho_arquivo':'data/raw/acidentes-2019.csv', 'colunas_alvo':mapeamento_2019},
    2020:{ 'caminho_arquivo':'data/raw/acidentes_2020-novo.csv', 'colunas_alvo':mapeamento_2020_2021},
    2021:{ 'caminho_arquivo':'data/raw/acidentes2021.csv', 'colunas_alvo':mapeamento_2020_2021},
    2022:{ 'caminho_arquivo':'data/raw/acidentes2022.csv', 'colunas_alvo':mapeamento_2022}   
}

In [5]:
pos_processamento = []
for ano in range(2019,2023,1):
    pos_processamento.append(processar_df(ano, pre_processamento[ano]['caminho_arquivo'], 
                                          pre_processamento[ano]['colunas_alvo'], 
                                          colunas_finais))
    

In [6]:
base_bruta = pd.concat(pos_processamento)


### Otimizando base

In [8]:
colunas_a_zerar = ['AUTOMOVEL_ENVOLVIDO', 'MOTO_ENVOLVIDA',
       'CICLO_MOTOR_ENVOLVIDO', 'CICLISTA_ENVOLVIDO', 'PEDESTRE_ENVOLVIDO',
       'ONIBUS_ENVOLVIDO', 'CAMINHAO_ENVOLVIDO', 'VIATURA_ENVOLVIDA',
       'OUTROS_ENVOLVIDO', 'NUM_VITIMAS_FATAIS']

In [9]:
base_bruta[colunas_a_zerar] = base_bruta[colunas_a_zerar].fillna(0)

In [10]:
base_removida = base_bruta.dropna()

### Processamento de data

In [60]:
def gerar_coluna_data(df, coluna_data):
    if coluna_data not in df.columns:
        print(f"A coluna '{coluna_data}' não existe no DataFrame.")
        return df
    
    df[coluna_data] = pd.to_datetime(df[coluna_data])
    df['DIA'] = df[coluna_data].dt.day
    df['MES'] = df[coluna_data].dt.month
    df['ANO'] = df[coluna_data].dt.year
    return df

In [ ]:
base_otimizada = gerar_coluna_data(base_removida, "DATA")

### Processamento de hora

In [83]:
def classificar_turnos(df, nome_coluna_hora):
    if nome_coluna_hora not in df.columns:
        print(f"A coluna '{nome_coluna_hora}' não existe no DataFrame.")
        return df
    
    # Remove registros com formato de hora inválido
    df = df[df[nome_coluna_hora].apply(verificar_hora_valida)]
    
    df[nome_coluna_hora] = pd.to_datetime(df[nome_coluna_hora])
    df['TURNO'] = df[nome_coluna_hora].apply(lambda x: classificar_turno(x.time() if isinstance(x, pd.Timestamp) else None))

    return df

def verificar_hora_valida(hora):
    try:
        pd.to_datetime(hora)
        return True
    except ValueError:
        return False

def classificar_turno(hora):
    if hora is None:
        return None

    hora_str = hora.strftime('%H:%M:%S')
    
    if '06:00:00' <= hora_str < '12:00:00':
        return 'MANHÃ'
    elif '12:00:00' <= hora_str < '18:00:00':
        return 'TARDE'
    elif '18:00:00' <= hora_str <= '23:59:59':
        return 'NOITE'
    else:
        return 'MADRUGADA'

In [84]:
base_otimizada = classificar_turnos(base_otimizada,'HORA')

C:\Users\abili\AppData\Local\Temp\ipykernel_9388\4250460889.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[nome_coluna_hora] = pd.to_datetime(df[nome_coluna_hora])
C:\Users\abili\AppData\Local\Temp\ipykernel_9388\4250460889.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[nome_coluna_hora] = pd.to_datetime(df[nome_coluna_hora])
C:\Users\abili\AppData\Local\Temp\ipykernel_9388\4250460889.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [85]:
base_otimizada

,DATA,HORA,NATUREZA_ACIDENTE,BAIRRO,BAIRRO_CRUZAMENTO,TIPO,AUTOMOVEL_ENVOLVIDO,MOTO_ENVOLVIDA,CICLO_MOTOR_ENVOLVIDO,CICLISTA_ENVOLVIDO,...,ONIBUS_ENVOLVIDO,CAMINHAO_ENVOLVIDO,VIATURA_ENVOLVIDA,OUTROS_ENVOLVIDO,NUM_VITIMAS,NUM_VITIMAS_FATAIS,DIA,MES,ANO,TURNO
0,2019-01-01,2023-08-20 00:41:00,SEM VÍTIMA,IPSEP,IPSEP,COLISÃO TRASEIRA,2.0,0,0,0,...,0,0,0,0,0.0,0,1,1,2019,MADRUGADA
1,2019-01-01,2023-08-20 01:37:00,SEM VÍTIMA,BOA VIAGEM,BOA VIAGEM,ABALROAMENTO TRANSVERSAL,2.0,0,0,0,...,0,0,0,0,0.0,0,1,1,2019,MADRUGADA
2,2019-01-01,2023-08-20 14:20:00,SEM VÍTIMA,BOA VIAGEM,BOA VIAGEM,COLISÃO,2.0,0,0,0,...,0,0,0,0,0.0,0,1,1,2019,TARDE
3,2019-01-01,2023-08-20 02:53:00,SEM VÍTIMA,IMBIRIBEIRA,IMBIRIBEIRA,COLISÃO,2.0,0,0,0,...,0,0,0,0,0.0,0,1,1,2019,MADRUGADA
4,2019-01-01,2023-08-20 08:17:00,COM VÍTIMA,JAQUEIRA,JAQUEIRA,COLISÃO COM CICLISTA,0,0,0,1.0,...,1.0,0,0,0,1.0,0,1,1,2019,MANHÃ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2375,2021-12-31,2023-08-20 11:36:00,COM VÍTIMA,ESTÂNCIA,ESTÂNCIA,COLISÃO TRASEIRA,1,1,0,0,...,0,0,0,0,1,0,31,12,2021,MANHÃ
2376,2021-12-31,2023-08-20 12:56:00,COM VÍTIMA,VÁRZEA,VÁRZEA,COLISÃO FRONTAL,0,1,0,0,...,1,0,0,0,1,0,31,12,2021,TARDE
2377,2021-12-31,2023-08-20 15:00:00,COM VÍTIMA,TORREÃO,TORREÃO,COLISÃO TRASEIRA,2,0,0,0,...,0,0,0,0,1,0,31,12,2021,TARDE
2378,2021-12-31,2023-08-20 20:44:00,COM VÍTIMA,PRADO,PRADO,COLISÃO FRONTAL,2,0,0,0,...,0,0,0,0,2,0,31,12,2021,NOITE


## Relatório final da base 

In [86]:
base_bruta.shape[0]

21203

In [87]:
base_otimizada.shape[0]

18225

In [88]:
base_otimizada.to_parquet('base_acidentes_processada')